In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import  mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import r2_score,mean_squared_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")


In [41]:
df = pd.read_csv("data/StudentsPerformance.csv")


In [42]:
x = df.drop('math score',axis = "columns")
y = df['math score']



In [43]:
num_feature = x.select_dtypes(exclude = "object").columns
cat_feature = x.select_dtypes(include = "object").columns
 

In [44]:
from sklearn.preprocessing import  OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
preprocessor = ColumnTransformer([
    ("OneHotEncoder",oh_transformer,cat_feature),
    ("StandardScaler",numeric_transformer,num_feature)
    ])

In [45]:
x = preprocessor.fit_transform(x)
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2,random_state = 42)
 


In [50]:
def evaluate_model(true,predicted):
    mae = mean_squared_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_scuare = r2_score(true,predicted)
    return mae,mse,rmse


In [53]:
models = {
    "LinearRegression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "KNeighbors Regressor" : KNeighborsRegressor(),
    "RandomForestRegressor" : RandomForestRegressor(),
    "XGBRegressor" : XGBRegressor(),
    "CatBoostRegressor" : CatBoostRegressor(),
    "AdaBoostRegressor" : AdaBoostRegressor(),


}
model_list = []
r2_list = []

for  i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    model_train_mae,model_train_rmse,model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2 = evaluate_model(y_test,y_test_pred)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model performence for training set :")
    print("Root mean squared error ", model_train_rmse)
    print("Mean absolute error ",model_train_mae)
    print("R2 score ",model_train_r2)
    print("............................")

    print("Model performance for test set")
    print("Root mean squared error ",model_test_rmse)
    print("Mean absolute error ",model_test_mae)
    print("r2 score ",model_test_r2)
    print("*" * 35)
    print("\n")


LinearRegression
Model performence for training set :
Root mean squared error  28.377889404296877
Mean absolute error  28.377889404296877
R2 score  5.3270901441872445
............................
Model performance for test set
Root mean squared error  29.3608056640625
Mean absolute error  29.3608056640625
r2 score  5.418561217155575
***********************************


Lasso
Model performence for training set :
Root mean squared error  43.47840400585579
Mean absolute error  43.47840400585579
R2 score  6.593815587795566
............................
Model performance for test set
Root mean squared error  42.5064168384116
Mean absolute error  42.5064168384116
r2 score  6.519694535667419
***********************************


Ridge
Model performence for training set :
Root mean squared error  28.337788233082435
Mean absolute error  28.337788233082435
R2 score  5.323324922741654
............................
Model performance for test set
Root mean squared error  29.056272192348285
Mean abso